# Doubly-sparse PCA

## Problem statement

This method is trying to solve the biclustering problem: given an $n \times p$ matrix with data X (where the data represents gene expression, e.g. $n$ is the number of  patients and $p$ is the number of genes with their expression measured), we are trying to find a principal component such, that it represents a subset of genes and a subset of patients for which the sum of square distances from data points to this principal component is minimal.

$X = \begin{pmatrix} x_{1,1} && x_{1,2} && ... && x_{1,p} \\ ... && ... && ... && ... \\ x_{n,1} && x_{n,2} && ... && x_{n,p} \end{pmatrix}$

Let ${\bf v} = (v_1, v_2, ..., v_p)^T$ be the first principal component vector, we are looking for. We want a $k$-sparse version of this vector (i.e. we want its $L_0$ norm to be $k$), so that only $k$ coordinates of it are non-zero. Unfortunately, with $L_0$ restriction, the problem is NP-hard, so instead of using $L_0$ norm, we will use $L_1$ norm as its proxy.

We would also consider only a subset of data points projections on this principal component. Again, to select the member data point of this subset, we need to apply $L_0$ norm as a restriction, but instead we will go for $L_1$ norm. The mask, meant to select just a subset of the points, would be represented by a vector ${\bf u} = (u_1, u_2, ..., u_n)^T$.

Thus, the function, we are seeking to optimize is:

$f(u, v) = \begin{pmatrix} v_1 && v_2 && v_3 && v_4 && v_5 \end{pmatrix} \begin{pmatrix} x_{1,1} && ... && x_{n,1} \\  x_{1,2} && ... && x_{n,2} \\ x_{1,3} && ... && x_{n,3} \\ ... && ... && ... \\ x_{1,p} && ... && x_{n,p} \end{pmatrix} \begin{pmatrix} u_1 && 0 && 0 \\ 0 && u_2 && 0 \\ 0 && 0 && u_n \end{pmatrix} \begin{pmatrix} x_{1,1} && x_{1,2} && x_{1,3} && ... && x_{1,p} \\ ... && ... && ... && ... && ... \\ x_{n,1} && x_{n,2} && x_{n,3} && ... && x_{n,p} \end{pmatrix} \begin{pmatrix} v_1 \\ v_2 \\ v_3 \\ ... \\ v_p \end{pmatrix} \to \min$

Upon conditions:

$\sum \limits_{i=1}^{p} | v_i | = k$

$\sum \limits_{i=1}^{n} | u_i | = s$

## Solution

Re-write the function in scalar way:

$f({\bf u}, {\bf v}) = \begin{pmatrix} \sum \limits_{i=1}^{n} v_i x_{1,i} && ... && \sum \limits_{i=1}^{n} v_i x_{n,i} \end{pmatrix} \begin{pmatrix} u_1 && 0 && 0 \\ 0 && u_2 && 0 \\ 0 && 0 && u_n \end{pmatrix} \begin{pmatrix} \sum \limits_{i=1}^p x_{1,i} v_i \\ ... \\ \sum \limits_{i=1}^p x_{n,i} v_i \end{pmatrix} = u_1 \cdot (\sum \limits_{i=1}^p x_{1,i} v_i)^2 + u_2 \cdot (\sum \limits_{i=1}^p x_{2,i} v_i)^2 + ... + u_n \cdot (\sum \limits_{i=1}^p x_{n,i} v_i)^2 = \sum \limits_{j=1}^n u_j (\sum \limits_{i=1}^p x_{j,i} v_i)^2$

Here we have $n$ variables $u_j$ and $p$ variables $v_i$. We need the partial derivatives of $f({\bf u}, {\bf v})$ on those variables to equal zero, and also two linear constraints to hold.

$\frac{\partial f}{\partial u_j} = (\sum \limits_{i=1}^p x_{j,i} v_i)^2$

$\frac{\partial f}{\partial v_i} = 2 u_1 (\sum \limits_{t=1}^p x_{1,t} v_t) x_{1,i} + 2 u_2 (\sum \limits_{t=1}^p x_{2,t} v_t) x_{2,i} + ... + 2 u_n (\sum \limits_{t=1}^p x_{n,t} v_t) x_{n,i}$

We have to add constraints with Lagrange multipliers to these equations (TODO: I discarded absolute values here! Fix it!):

$ -\lambda_1 \sum \limits_{i=1}^p v_i = -\lambda_1 k$

$ -\lambda_2 \sum \limits_{j=1}^p u_j = -\lambda_2 s$

Hence, we get a set of $p + n + 2$ equations:

$\frac{\partial (f + constraints)}{\partial u_j} = (\sum \limits_{i=1}^p x_{j,i} v_i)^2 - \lambda_2 \cdot sign(u_j) = 0$

$\frac{\partial (f + constraints)}{\partial v_i} = 2 u_1 (\sum \limits_{t=1}^p x_{1,t} v_t) x_{1,i} + 2 u_2 (\sum \limits_{t=1}^p x_{2,t} v_t) x_{2,i} + ... + 2 u_n (\sum \limits_{t=1}^p x_{n,t} v_t) x_{n,i} - \lambda_1 \cdot sign(v_i) = 0$

$\sum \limits_{i=1}^p | v_i | = k$

$\sum \limits_{j=1}^p | u_j |= s$

$
\begin{equation}
  \begin{cases}
    (\sum \limits_{i=1}^p x_{j,i} v_i)^2 = \lambda_2 \cdot sign(u_j) \\
    2 u_1 (\sum \limits_{t=1}^p x_{1,t} v_t) x_{1,i} + 2 u_2 (\sum \limits_{t=1}^p x_{2,t} v_t) x_{2,i} + ... + 2 u_n (\sum \limits_{t=1}^p x_{n,t} v_t) x_{n,i} = \lambda_1 \cdot sign(v_i) \\
    \sum \limits_{i=1}^p | v_i | = k \\
    \sum \limits_{j=1}^p | u_j | = s \\
  \end{cases}
\end{equation}
$

In [5]:
def doubly_sparse_pca_sum_of_squared_distances(X, v, u):
    """Returns the square sum of distances from a subset of datapoints X, masked by vector u,
    to the first sparse principal component axis, determined by vector v.
    
    :param 2D-ndarray X: n-by-p (n rows, p columns) data matrix, where n 
      correspond to data vectors (e.g. RNA-seq'ed human) and p correspond to
      predictors (e.g. gene expressions);
    :param ndarray v: L1-norm-limited first principal component vector; it is
      a sparse p-vector that selects a subset of predictors to be non-zero,
      while the remaining predictors are zero;
    :param ndarray u: L1-norm-limited mask that selects a subset of datapoints,
      for which we minimize the sum of square distances to the first principal
      component vector; it is a sparse n-vector
    :raise: ValueError if len(v) != len(X) or if len(u) != len(X[0]) or if X/v/u are empty
    :return: sum of squared distances from the selected subset of vectors to
      the first principal component
    :rtype: float
    """
    if len(X) == 0:
        raise ValueError(f"len(X) == 0")
    
    if len(X[0]) == 0:
        raise ValueError(f"len(X[0]) == 0")
    
    if len(v) == len(X):
        raise ValueError(f"len(v) != len(X): len(v) = {len(v)}, len(X) = {len(X)}")
        
    if len(u) == len(X[0]):
        raise ValueError(f"len(u) != len(X[0]): len(u) = {len(u)}, len(X[0]) = {len(X[0])}")
        
    sum: float = 0
    for j, u_j in enumerate(u):
        squared_distance: float = 0
        for i, v_i in enumerate(v):
            squared_distance += X[j][i] * v_i  # TODO: refactor as a 
        sum += squared_distance

    return sum


def double_sparse_pca_sum_of_sqared_distances_derivatives():
    pass

In [ ]:
import scipy
from scipy.optimize import LinearConstraint, minimize


linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

res = minimize(
    doubly_sparse_pca_sum_of_squared_distances, 
    X,
    method='SLSQP',
    jac=rosen_der,
    constraints=[inequality_constraints], 
    options={'ftol': 1e-9, 'disp': True},
    bounds=bounds
)

In [ ]:
import cvxopt


